In [53]:
import pandas as pd

In [54]:
swift_df = pd.read_csv('Swift Codes.csv')
swift_df.head(10)

,Bank,SWIFT code,Check Digits
0,Lloyds Bank,LOYD,C1
1,Barclays Bank,BARC,22
2,Halifax,HLFX,22
3,HSBC,HBUK,4B
4,Natwest,NWBK,2L
5,Santander,ABBY,3E
6,Data Source Bank,DSBX,12


In [55]:
trans_df = pd.read_csv('Transactions.csv')
trans_df.head(5)

,Transaction ID,Account Number,Sort Code,Bank
0,3888,62230725,95-98-82,Data Source Bank
1,4746,83172326,42-86-38,Barclays Bank
2,5404,34302539,53-28-21,Barclays Bank
3,9013,13350031,93-87-71,Natwest
4,2535,68745993,57-14-32,Barclays Bank


In [56]:
# In the Transactions table, there is a Sort Code field which contains dashes. We need to remove these so just have a 6 digit string

trans_df['Sort Code'] = [x.replace('-', '') for x in trans_df['Sort Code']]
trans_df.head(5)

,Transaction ID,Account Number,Sort Code,Bank
0,3888,62230725,959882,Data Source Bank
1,4746,83172326,428638,Barclays Bank
2,5404,34302539,532821,Barclays Bank
3,9013,13350031,938771,Natwest
4,2535,68745993,571432,Barclays Bank


In [57]:
# Use the SWIFT Bank Code lookup table to bring in additional information about the SWIFT code and Check Digits of the receiving bank account

trans_df = trans_df.merge(swift_df, on='Bank', how='left')
trans_df.head(5)

,Transaction ID,Account Number,Sort Code,Bank,SWIFT code,Check Digits
0,3888,62230725,959882,Data Source Bank,DSBX,12
1,4746,83172326,428638,Barclays Bank,BARC,22
2,5404,34302539,532821,Barclays Bank,BARC,22
3,9013,13350031,938771,Natwest,NWBK,2L
4,2535,68745993,571432,Barclays Bank,BARC,22


In [58]:
# Add a field for the Country Code

trans_df['Country Code'] = 'GB'
trans_df.head(5)

,Transaction ID,Account Number,Sort Code,Bank,SWIFT code,Check Digits,Country Code
0,3888,62230725,959882,Data Source Bank,DSBX,12,GB
1,4746,83172326,428638,Barclays Bank,BARC,22,GB
2,5404,34302539,532821,Barclays Bank,BARC,22,GB
3,9013,13350031,938771,Natwest,NWBK,2L,GB
4,2535,68745993,571432,Barclays Bank,BARC,22,GB


In [59]:
# Create the IBAN as above
trans_df['IBAN'] = ''
for i in range(len(trans_df)):
    x = trans_df.loc[i,:].values.flatten().tolist()
    trans_df.iat[i,7] = f'{x[6]}{x[5]}{x[4]}{x[2]}{x[1]}'
trans_df.head(5)

,Transaction ID,Account Number,Sort Code,Bank,SWIFT code,Check Digits,Country Code,IBAN
0,3888,62230725,959882,Data Source Bank,DSBX,12,GB,GB12DSBX95988262230725
1,4746,83172326,428638,Barclays Bank,BARC,22,GB,GB22BARC42863883172326
2,5404,34302539,532821,Barclays Bank,BARC,22,GB,GB22BARC53282134302539
3,9013,13350031,938771,Natwest,NWBK,2L,GB,GB2LNWBK93877113350031
4,2535,68745993,571432,Barclays Bank,BARC,22,GB,GB22BARC57143268745993


In [60]:
# Remove unnecessary fields
trans_df = trans_df.drop(trans_df.iloc[:,1:-1] , axis=1)
trans_df.head(5)

,Transaction ID,IBAN
0,3888,GB12DSBX95988262230725
1,4746,GB22BARC42863883172326
2,5404,GB22BARC53282134302539
3,9013,GB2LNWBK93877113350031
4,2535,GB22BARC57143268745993


In [62]:
trans_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Transaction ID  100 non-null    int64 
 1   IBAN            100 non-null    object
dtypes: int64(1), object(1)
memory usage: 6.4+ KB


In [61]:
# Write output to CSV file

trans_df.to_csv('output.csv', index=False)